# FactSet Sample Codes

## Database Setup

This part connects Python to the FactSet database. You don't need to change the codes. You may need to install required packages as neccessary.

A prerequisite before connectiong to the dataset is ODBC SQL driver. 
Install the driver following <https://learn.microsoft.com/en-us/sql/connect/odbc/linux-mac/install-microsoft-odbc-driver-sql-server-macos?view=sql-server-ver16>

Note that if the database can't find the ODBS driver, need to create symbolic links to find registered drivers.

```bash
sudo ln -s /usr/local/etc/odbcinst.ini /etc/odbcinst.ini
sudo ln -s /usr/local/etc/odbc.ini /etc/odbc.ini
```

In [4]:
# install required packages
import sys
!{sys.executable} -m pip install adal
!{sys.executable} -m pip install pyodbc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 855.4 kB/s eta 0:00:00 0:00:01


In [1]:
import sys
import pyodbc
import requests
import json
import struct
import adal
import functools
import pandas as pd

In [8]:
# static values for database connection
authentication_endpoint = 'https://login.microsoftonline.com/' # standard endpoint prefix for all Azure AD OAuth requests
resource  = 'https://database.windows.net/' # the trainling / is required without it you will get a token but SQL Auth will fail
driver= '{ODBC Driver 18 for SQL Server}' # required for connection string
# end static values

# Enter Application (SQL Server) information
server = 'ofprod-norduni-1214693-sql.database.windows.net'  # the server where your database lives
database = 'fds'                                           # the database you wish to hit
application_id = '175bb48c-3918-4882-864d-e0005a3fb1b9'     # Application ID
application_secret = 's*adq9nuPyIlO6?6'                    # Application Secret
tenant_id = '19c2ed02-03ce-49b9-ab35-7e07b729dc04'          # Tenant ID
# end Application information


_last_args = {}
def defaults_from_globals(f):
    @functools.wraps(f)
    def wrapper(*args, **kwargs):
        _last_args.update(kwargs)
        ret =  f(*args, **_last_args)
        return ret
    return wrapper


@defaults_from_globals
def run_query_on_database(query=None, server=server, database=database, application_id=application_id, application_secret=application_secret, tenant_id=tenant_id):
    try:
        # auth and connection setup
        context = adal.AuthenticationContext(authentication_endpoint + tenant_id)
        token_response = context.acquire_token_with_client_credentials(resource, application_id, application_secret)
        token = token_response.get('accessToken')
        exptoken = token.encode("utf_16_le")
        tokenstruct = struct.pack("=i", len(exptoken)) + exptoken
        connstr = 'DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database
        cnxn = pyodbc.connect(connstr, attrs_before = { 1256:bytearray(tokenstruct) })
        # end auth and connection setup
        
        df_sql = pd.read_sql(query, cnxn)
        cnxn.close()
        return (df_sql)

    except Exception as e:
        error_msg = "Encountered error while running query on {server}\{database} - Error was:\n {error}".format(server=server, database=database,error=e)
        print(error_msg)



## Start your queries and analysis here

<font size="4">Write and run you SQL queries, output to a csv file.</font>

In [10]:
# test query one 
query1 = 'SELECT TOP 20 * FROM sym_v1.sym_ticker_exchange'
price_df_sql = run_query_on_database(query=query1)
price_df_sql

/var/folders/y7/fctfc2vx5cq235qtnlxb211w0000gn/T/ipykernel_17471/1169710532.py:39: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sql = pd.read_sql(query, cnxn)


,fsym_id,ticker_exchange
0,B00001-L,DE000TT91W34-MUNC
1,B00012-L,58537.XX9-HKG
2,B00022-L,E680S.XX7-PAR
3,B00033-L,B00033.W-STU
4,B00045-L,DE000HB6EL29-MUNC
5,B00056-L,DE000PE5PEE7-BNP
6,B00066-L,Q983S.XX6-PAR
7,B00077-L,B00077.W-STU
8,B00088-L,A325G-PAR
9,B00089-L,B00089.W-STU


In [11]:
# test query two
ticker_list = ['AAPL-US','IBM-US','SPY-US','MSFT-US','GOOGL-US'] 
ticker_string = ','.join(map("'{0}'".format, ticker_list))
query2 = '''SELECT price.*,p.one_day_pct, sym.ticker_region, cov.proper_name, isin.isin
                                FROM fp_v2.fp_total_returns_daily AS p
                                LEFT JOIN fp_v2.fp_basic_prices AS price
                                ON price.fsym_id = p.fsym_id AND p.p_date = price.p_date
                                LEFT JOIN sym_v1.sym_ticker_region AS sym
                                ON p.fsym_id = sym.fsym_id
                                JOIN sym_v1.sym_coverage AS cov
                                ON sym.fsym_id = cov.fsym_id
                                LEFT JOIN sym_v1.sym_isin AS isin
                                ON  cov.fsym_security_id = isin.fsym_id
    WHERE sym.ticker_region IN ({})
    AND p.p_date >= '2021-08-16'
      ORDER BY p.p_date, sym.ticker_region'''.format(ticker_string)
price_df_sql = run_query_on_database(query=query2)
price_df_sql

/var/folders/y7/fctfc2vx5cq235qtnlxb211w0000gn/T/ipykernel_17471/1169710532.py:39: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sql = pd.read_sql(query, cnxn)


,fsym_id,p_date,currency,p_price,p_price_open,p_price_high,p_price_low,p_volume,one_day_pct,ticker_region,proper_name,isin
0,MH33D6-R,2021-08-16,USD,151.12,148.535,151.19,146.4700,103558.797,1.354790,AAPL-US,Apple Inc.,US0378331005
1,HTM0LK-R,2021-08-16,USD,2766.19,2750.498,2767.25,2705.9750,1119.918,0.422561,GOOGL-US,Alphabet Inc. Class A,US02079K3059
2,SJY281-R,2021-08-16,USD,143.59,143.230,143.74,142.2300,2786.343,0.286353,IBM-US,International Business Machines Corporation,US4592001014
3,P8R3C2-R,2021-08-16,USD,294.60,293.190,294.82,290.0200,22516.029,0.597572,MSFT-US,Microsoft Corporation,US5949181045
4,M75BNK-R,2021-08-16,USD,446.97,444.530,447.11,442.8700,73930.867,0.235462,SPY-US,SPDR S&P 500 ETF Trust,US78462F1030
...,...,...,...,...,...,...,...,...,...,...,...,...
3270,MH33D6-R,2024-02-16,USD,182.31,183.420,184.85,181.6650,49752.473,-0.843036,AAPL-US,Apple Inc.,US0378331005
3271,HTM0LK-R,2024-02-16,USD,140.52,142.990,143.19,140.1400,31468.930,-1.575960,GOOGL-US,Alphabet Inc. Class A,US02079K3059
3272,SJY281-R,2024-02-16,USD,187.64,186.630,188.95,185.9452,4842.840,0.412059,IBM-US,International Business Machines Corporation,US4592001014
3273,P8R3C2-R,2024-02-16,USD,404.06,407.960,408.29,403.4400,22296.500,-0.614917,MSFT-US,Microsoft Corporation,US5949181045


In [5]:
# write to csv
f_name = "price_df.csv"
price_df_sql.to_csv(f_name, index=False)
print ("Output to file: {}".format(f_name))

Output to file: price_df.csv
